In [1]:
import h2o_mlops_scoring_client
import os

In [2]:
num_cores = 10
h2o_mlops_scoring_client.spark_master = f"local[{num_cores}]"

In [3]:
h2o_mlops_scoring_client.spark_conf_dir = os.path.expanduser("/Users/kishani/h2oapps/Joe MLOps Scoring Test/conf")


In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.expanduser("/Users/kishani/h2oapps/Joe MLOps Scoring Test/kishani-mlops-scoring-test-2461a12de86c.json")

In [5]:
MLOPS_ENDPOINT_URL = "https://model.cloud-qa.h2o.ai/91427651-b54e-4e76-83e2-e67552931f4c/model/score"

In [6]:
ID_COLUMN = "ID"
SOURCE_DATA = "kishani-mlops-scoring-test.gbm_dataset.test_table"
SINK_LOCATION = "kishani-mlops-scoring-test.gbm_dataset.mlops_scoring_client"

In [7]:
SOURCE_FORMAT = h2o_mlops_scoring_client.Format.BIGQUERY
SINK_FORMAT = h2o_mlops_scoring_client.Format.BIGQUERY

In [8]:
h2o_mlops_scoring_client.WriteMode.OVERWRITE.value

'Overwrite existing files'

In [9]:
SINK_WRITE_MODE = h2o_mlops_scoring_client.WriteMode.OVERWRITE

In [10]:
def preprocess(spark_df):
    return spark_df.repartition(30)

In [11]:
from pyspark.sql.functions import col

def postprocess(spark_df):
    """Required for GBQ."""
    sanitized_column_names = {}
    for c in spark_df.columns:
        if "." in c:
            sanitized_column_names[c] = c.replace(".", "_")
    return spark_df.select(
        [col(f"`{c}`").alias(sanitized_column_names.get(c, c)) for c in spark_df.columns]
    )

In [12]:
h2o_mlops_scoring_client.score_source_sink(
    mlops_endpoint_url=MLOPS_ENDPOINT_URL,
    id_column=ID_COLUMN,
    source_data=SOURCE_DATA,
    source_format=SOURCE_FORMAT,
    sink_location=SINK_LOCATION,
    sink_format=SINK_FORMAT,
    sink_write_mode=SINK_WRITE_MODE,
    preprocess_method=preprocess,
    postprocess_method=postprocess,
    spark_config_overrides={
        "temporaryGcsBucket": "test-bucket-mlops-scoring-client"
    }
) 

23/05/04 00:10:53 INFO h2o_mlops_scoring_client: Starting Spark context
23/05/04 00:10:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/04 00:10:58 INFO h2o_mlops_scoring_client: Connecting to H2O.ai MLOps scorer at 'https://model.cloud-qa.h2o.ai/91427651-b54e-4e76-83e2-e67552931f4c/model/score'
23/05/04 00:11:02 INFO h2o_mlops_scoring_client: Applying preprocess method
23/05/04 00:11:02 INFO h2o_mlops_scoring_client: Starting scoring from 'kishani-mlops-scoring-test.gbm_dataset.test_table' to 'kishani-mlops-scoring-test.gbm_dataset.mlops_scoring_client'
23/05/04 00:11:04 INFO h2o_mlops_scoring_client: Applying postprocess method
23/05/04 00:11:07 INFO h2o_mlops_scoring_client: Stopping Spark context


Py4JJavaError: An error occurred while calling o110.save.
: java.io.IOException: Error accessing gs://test-bucket-mlops-scoring-client/.spark-bigquery-local-1683139257638-a89378a6-3775-4045-ba97-29119c14343f
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getObject(GoogleCloudStorageImpl.java:2233)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getItemInfo(GoogleCloudStorageImpl.java:2120)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfoInternal(GoogleCloudStorageFileSystem.java:1104)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfo(GoogleCloudStorageFileSystem.java:1078)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.getFileStatus(GoogleHadoopFileSystemBase.java:958)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1760)
	at com.google.c